# SQL——Structured Query Language


结构化查询语言SQL（STRUCTURED QUERY LANGUAGE）是最重要的关系数据库操作语言，并且它的影响已经超出数据库领域，得到其他领域的重视和采用，如人工智能领域的数据检索，第四代软件开发工具中嵌入SQL的语言等。


## Basic query in MySQL
###1. CREATE TABLE
create table table_name

(column1 data type(NULL||NOT NULL));

desc: create a table
 
###2. SHOW table
show CREATE TABLE table_name;

desc: show the table in your database
###3. INSERT
insert into table_name

values('value1','value2',......);

desc: insert new rows of data into the database

###4. SELECT 
select * from table table_name.

desc: return data from the table

###5. SELECT
select column1,column2,... 

from table1,table2...

[where conditions]

[group by column1,column2...]

[having conditions]

[order by column1,column2...]

desc: return data from one or more database tables

###6. UPDATE
update table_name set

column1='value1,column2='value2'...

[where conditions]

desc: update existing data in the table

### 7. DELETE
delete from table_name

[where conditions]

desc: delete rows of data from a table


##MySQL class
###1. SELECT
select *

select col1,col2...

select distinct(column)

select count(*)

 desc: define columns to display as part of query output

### 2. FROM
from table1,table2,table3...

desc: defines tables from which to retrieve data

### 3. GROUP BY
group by grp_col1,grp_col2...

desc: a form of a sorting used to devide output into logical groups

### 4. HAVING
having grp_col1='value1'

AND grp_col2='value2''

desc: similar to the where clause, used to place conditions on the group clause

### 5. ORDER BY
order by col1,col2...

desc: used to sort a query's result


## MySQL Functions

###1. rand()

desc: return random values

###2. sum()

desc: return the sum of the values

###3. now()

desc: return the time an date of the system

###4. count()

desc: return the count of the values which satisfy the given condition
